In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [2]:
load_dotenv()

True

In [3]:
api_key=os.getenv("MY_KEY")

In [4]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key,temperature=0.5)

In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions and options are unique, not repeated, and varied in structure. Ensure that each question is distinct and tailored to different parts of the content.
Ensure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} unique MCQs.
### RESPONSE_JSON
{response_json}
"""


In [8]:

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [9]:
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions and options are unique, not repeated, and varied in structure. Ensure that each question is distinct and tailored to different parts of the content.\nEnsure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} unique MCQs.\n### RESPONSE_JSON\n{response_json}\n')

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\Admin\SelfContentMCQGen\env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [11]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions and options are unique, not repeated, and varied in structure. Ensure that each question is distinct and tailored to different parts of the content.\nEnsure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} unique MCQs.\n### RESPONSE_JSON\n{response_json}\n'), llm=GoogleGenerativeAI(model='models/text-bison-001', google_api_key=SecretStr('**********'), temperature=0.5, client=<module 'google.generativeai' from 'c:\\Users\\Admin\\SelfContentMCQGen\\env\\lib\\site-packages\\google\\generativeai\\__init__.py'>), output_key='quiz')

In [12]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [14]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [15]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain], 
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"], 
    verbose=True
)

In [16]:
"""pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()"""

'pdf_reader=PyPDF2.PdfReader(r"C:\\Complete_Content\\All_Project\\TEST_FOR_EVERYTHING\\langchain\\experiments\\Big Data Assessment.pdf")\ntext = ""\nfor page in pdf_reader.pages:\n    text += page.extract_text()'

In [17]:
from pathlib import Path
file_path=r"C:\\Users\\Admin\\SelfContentMCQGen\\test.txt"

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [21]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [22]:
response = generate_evaluate_chain.invoke(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of 5 multiple choice questions for data science students in simple tone. 
Make sure the questions and options are unique, not repeated, and varied in structure. Ensure that each question is distinct and tailored to different parts of the content.
Ensure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make 5 unique MCQs.
### RESPONSE_JSON
{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice

In [23]:
quiz_str=response.get("quiz")
quiz_str

'```json\n{\n  "1": {\n    "mcq": "What is the first step of the CRISP-DM process?",\n    "options": {\n      "a": "Define the problem",\n      "b": "Gather data",\n      "c": "Clean data",\n      "d": "Analyze data"\n    },\n    "correct": "a"\n  },\n  "2": {\n    "mcq": "Which of the following is not a type of machine learning algorithm?",\n    "options": {\n      "a": "Supervised learning",\n      "b": "Unsupervised learning",\n      "c": "Reinforcement learning",\n      "d": "Natural language processing"\n    },\n    "correct": "d"\n  },\n  "3": {\n    "mcq": "What is the most common type of data used in machine learning?",\n    "options": {\n      "a": "Numerical data",\n      "b": "Categorical data",\n      "c": "Text data",\n      "d": "Images"\n    },\n    "correct": "a"\n  },\n  "4": {\n    "mcq": "Which of the following is not a metric used to evaluate the performance of a machine learning model?",\n    "options": {\n      "a": "Accuracy",\n      "b": "Precision",\n      "c":

In [26]:
quiz_str = quiz_str.strip("```json").strip("```")

quiz_dict = json.loads(quiz_str)

In [27]:
quiz_dict

{'1': {'mcq': 'What is the first step of the CRISP-DM process?',
  'options': {'a': 'Define the problem',
   'b': 'Gather data',
   'c': 'Clean data',
   'd': 'Analyze data'},
  'correct': 'a'},
 '2': {'mcq': 'Which of the following is not a type of machine learning algorithm?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised learning',
   'c': 'Reinforcement learning',
   'd': 'Natural language processing'},
  'correct': 'd'},
 '3': {'mcq': 'What is the most common type of data used in machine learning?',
  'options': {'a': 'Numerical data',
   'b': 'Categorical data',
   'c': 'Text data',
   'd': 'Images'},
  'correct': 'a'},
 '4': {'mcq': 'Which of the following is not a metric used to evaluate the performance of a machine learning model?',
  'options': {'a': 'Accuracy',
   'b': 'Precision',
   'c': 'Recall',
   'd': 'Mean squared error'},
  'correct': 'd'},
 '5': {'mcq': 'Which of the following is not a common challenge faced when working with machine learning data?

In [28]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [29]:
quiz_table_data

[{'MCQ': 'What is the first step of the CRISP-DM process?',
  'Choices': 'a: Define the problem | b: Gather data | c: Clean data | d: Analyze data',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is not a type of machine learning algorithm?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Natural language processing',
  'Correct': 'd'},
 {'MCQ': 'What is the most common type of data used in machine learning?',
  'Choices': 'a: Numerical data | b: Categorical data | c: Text data | d: Images',
  'Correct': 'a'},
 {'MCQ': 'Which of the following is not a metric used to evaluate the performance of a machine learning model?',
  'Choices': 'a: Accuracy | b: Precision | c: Recall | d: Mean squared error',
  'Correct': 'd'},
 {'MCQ': 'Which of the following is not a common challenge faced when working with machine learning data?',
  'Choices': 'a: Bias | b: Variance | c: Overfitting | d: Underfitting',
  'Correct': 'd'}]

In [30]:
df=pd.DataFrame(quiz_table_data)

In [31]:
df.to_csv("Data_Science_Quiz.csv",index=False)